# Building a Multi-Agent System(MAS) with MCP

Copyright 2025-2026, Denis Rothman

In this notebook we build a multi agent system from scratch. We will use an educational version of the Model Context Protocol for our agents to communicate. You will learn how to define a standard message format. You will create specialized AI agents like a researcher and a writer. You will also build an orchestrator to manage their workflow. Following the steps you will see how these parts work together. We will turn a high level goal into a finished product like a blog post. This gives you a direct understanding of the core ideas behind collaborative AI systems.

In [1]:
#@title Installing OpenAI and activating the API key
!pip install openai==2.14.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 2.12.0
    Uninstalling openai-2.12.0:
      Successfully uninstalled openai-2.12.0


In [2]:
# Imports and API Key Setup
# We will use the OpenAI library to interact with the LLM and Google Colab's
# secret manager to securely access your API key.

import os
from openai import OpenAI
from google.colab import userdata

# Load the API key from Colab secrets, set the env var, then init the client
try:
    api_key = userdata.get("API_KEY")
    if not api_key:
        raise userdata.SecretNotFoundError("API_KEY not found.")

    # Set environment variable for downstream tools/libraries
    os.environ["OPENAI_API_KEY"] = api_key

    # Create client (will read from OPENAI_API_KEY)
    client = OpenAI()
    print("OpenAI API key loaded and environment variable set successfully.")

except userdata.SecretNotFoundError:
    print('Secret "API_KEY" not found.')
    print('Please add your OpenAI API key to the Colab Secrets Manager.')
except Exception as e:
    print(f"An error occurred while loading the API key: {e}")


OpenAI API key loaded and environment variable set successfully.


In [3]:
#@title 1.Initializing the Client
# ------------------------------------------------------------------------------
# We'll need the `openai` library to communicate with the LLM.
# Note: This notebook assumes you have already run a setup cell in your Colab
# environment to load your API key from Colab Secrets into an environment
# variable, as you specified.
# ------------------------------------------------------------------------------

import json

# --- Initialize the OpenAI Client ---
# The client will automatically read the OPENAI_API_KEY from your environment.
client = OpenAI()
print("OpenAI client initialized.")

OpenAI client initialized.


In [4]:
#@title 2.Defining the Protocol: The MCP Standard
# ------------------------------------------------------------------------------
# Before we build our agents, we must define the language they will speak.
# MCP provides a simple, structured way to pass context. For this example,
# our MCP message will be a Python dictionary with key fields.
# ------------------------------------------------------------------------------
def create_mcp_message(sender, content, metadata=None):
    """Creates a standardized MCP message."""
    return {
        "protocol_version": "1.0",
        "sender": sender,
        "content": content,
        "metadata": metadata or {}
    }

print("--- Example MCP Message (Our Simplified Version) ---")
example_mcp = create_mcp_message(
    sender="Orchestrator",
    content="Research the benefits of the Mediterranean diet.",
    metadata={"task_id": "T-123", "priority": "high"}
)
print(json.dumps(example_mcp, indent=2))

--- Example MCP Message (Our Simplified Version) ---
{
  "protocol_version": "1.0",
  "sender": "Orchestrator",
  "content": "Research the benefits of the Mediterranean diet.",
  "metadata": {
    "task_id": "T-123",
    "priority": "high"
  }
}


In [5]:
#@title 3.Building the Agents: The Specialists
# ------------------------------------------------------------------------------
# Each agent is a function that takes an MCP message as input and returns one
# as output. The core of each agent is a carefully crafted "Semantic Blueprint"
# in the system prompt that defines its persona and task.
# ------------------------------------------------------------------------------

def call_llm(system_prompt, user_content):
    """A helper function to call the OpenAI API using the new client syntax."""
    try:
        # Using the updated client.chat.completions.create method
        response = client.chat.completions.create(
            model="gpt-5.2",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_content}
            ]
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"An error occurred with the API call: {e}"

# --- Agent 1: The Researcher ---
def researcher_agent(mcp_input):
    """
    This agent takes a research topic, finds information, and returns a summary.
    """
    print("\n[Researcher Agent Activated]")
    simulated_database = {
        "mediterranean diet": "The Mediterranean diet is rich in fruits, vegetables, whole grains, olive oil, and fish. Studies show it is associated with a lower risk of heart disease, improved brain health, and a longer lifespan. Key components include monounsaturated fats and antioxidants."
    }
    research_topic = mcp_input['content']
    research_result = simulated_database.get(research_topic.lower(), "No information found on this topic.")
    system_prompt = "You are a research analyst. Your task is to synthesize the provided information into 3-4 concise bullet points. Focus on the key findings."
    summary = call_llm(system_prompt, research_result)
    print(f"Research summary created for: '{research_topic}'")
    return create_mcp_message(
        sender="ResearcherAgent",
        content=summary,
        metadata={"source": "Simulated Internal DB"}
    )

# --- Agent 2: The Writer ---
def writer_agent(mcp_input):
    """
    This agent takes research findings and writes a short blog post.
    """
    print("\n[Writer Agent Activated]")
    research_summary = mcp_input['content']
    system_prompt = "You are a skilled content writer for a health and wellness blog. Your tone is engaging, informative, and encouraging. Your task is to take the following research points and write a short, appealing blog post (approx. 150 words) with a catchy title."
    blog_post = call_llm(system_prompt, research_summary)
    print("Blog post drafted.")
    return create_mcp_message(
        sender="WriterAgent",
        content=blog_post,
        metadata={"word_count": len(blog_post.split())}
    )


In [6]:
#@title 4.Building the Orchestrator: The Project Manager
# ------------------------------------------------------------------------------
# The Orchestrator manages the workflow. It calls the agents in the correct
# order, passing context from one to the next using MCP messages.
# ------------------------------------------------------------------------------

def orchestrator(initial_goal):
    """
    Manages the multi-agent workflow to achieve a high-level goal.
    """
    print("="*50)
    print(f"[Orchestrator] Goal Received: '{initial_goal}'")
    print("="*50)

    # --- Step 1: Orchestrator plans and calls the Researcher Agent ---
    print("\n[Orchestrator] Task 1: Research. Delegating to Researcher Agent.")
    research_topic = "Mediterranean Diet"
    mcp_to_researcher = create_mcp_message(
        sender="Orchestrator",
        content=research_topic
    )
    mcp_from_researcher = researcher_agent(mcp_to_researcher)
    print("\n[Orchestrator] Research complete. Received summary:")
    print("-" * 20)
    print(mcp_from_researcher['content'])
    print("-" * 20)

    # --- Step 2: Orchestrator calls the Writer Agent ---
    print("\n[Orchestrator] Task 2: Write Content. Delegating to Writer Agent.")
    mcp_to_writer = create_mcp_message(
        sender="Orchestrator",
        content=mcp_from_researcher['content']
    )
    mcp_from_writer = writer_agent(mcp_to_writer)
    print("\n[Orchestrator] Writing complete.")

    # --- Step 3: Orchestrator presents the final result ---
    final_output = mcp_from_writer['content']
    print("\n" + "="*50)
    print("[Orchestrator] Workflow Complete. Final Output:")
    print("="*50)
    print(final_output)

In [7]:
#@title 5.Run the System
# ------------------------------------------------------------------------------
# Let's give our Orchestrator a high-level goal and watch the agent team work.
# ------------------------------------------------------------------------------
# Note: the goal is hard coded for this exmaple before we make it a variable in the subsequent notebooks
user_goal = "Create a blog post about the benefits of the Mediterranean diet."
orchestrator(user_goal)

[Orchestrator] Goal Received: 'Create a blog post about the benefits of the Mediterranean diet.'

[Orchestrator] Task 1: Research. Delegating to Researcher Agent.

[Researcher Agent Activated]
Research summary created for: 'Mediterranean Diet'

[Orchestrator] Research complete. Received summary:
--------------------
- The Mediterranean diet emphasizes fruits, vegetables, whole grains, olive oil, and fish.  
- Research links it to a lower risk of heart disease and a longer lifespan.  
- Studies also associate it with improved brain health.  
- Benefits are attributed in part to monounsaturated fats and antioxidants.
--------------------

[Orchestrator] Task 2: Write Content. Delegating to Writer Agent.

[Writer Agent Activated]
Blog post drafted.

[Orchestrator] Writing complete.

[Orchestrator] Workflow Complete. Final Output:
## Eat Like the Mediterranean: A Simple Recipe for a Healthier Heart and Brain

If you’re looking for an eating style that feels satisfying *and* supports long-t